In [1]:
# https://datamarket.com/data/set/22ox/monthly-milk-production-pounds-per-cow-jan-62-dec-75#!ds=22ox&display=line
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, TimeDistributed, Reshape, Flatten, Dropout, BatchNormalization
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from copy import copy
from tensorflow.keras import backend as K
import pandas as pd
from sklearn import preprocessing
import tensorflow.keras.optimizers as optimizers


In [2]:
train_split_size = 0.65
lookback_window = 150  # months.
milk = pd.read_csv('monthly-milk-production-pounds-p.csv', index_col=0, parse_dates=True)
labels = pd.read_csv('../data/dengue_labels_train.csv', parse_dates=True)
features = pd.read_csv('../data/dengue_features_train.csv', parse_dates=True)

all_data = pd.merge(labels, features, on=['city', 'year', 'weekofyear'])
sj_data = all_data[all_data.city == 'sj']
df = sj_data.drop(['city', 'week_start_date', 'year', 'weekofyear'], axis=1)


df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

In [3]:
# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# min_max_scaler.fit(x)
# x_scaled = min_max_scaler.transform(x)
# df = pd.DataFrame(x_scaled, columns=df.columns)

## Separate splits and perform data augmentation

In [4]:
def format_data(df, lookback_window=12):
    x = []
    y = []
    for i in range(len(df)-lookback_window):
        x_entry = copy(df[i:i+lookback_window+1].values.T) # include current value
        x_entry[0][lookback_window] = 0.0 # erase variable to be predicted
        x.append(x_entry)
        y_entry = df['total_cases'][i+lookback_window]
        y.append(y_entry)
        
    x = np.array(x)
    y = np.array(y)
    return x, y
    

x, y = format_data(df, lookback_window)

print(x.shape)
print(y.shape)

(786, 21, 151)
(786,)


In [5]:
def split(x, y, train_split_size):
    train_size = int(math.ceil(len(df)*train_split_size))
    val_size = int(math.floor((len(df)*(1-train_split_size))))
    
    x_train, y_train = x[0:train_size], y[0:train_size]
    x_val, y_val = x[train_size:train_size+val_size], y[train_size:train_size+val_size]
    
    assert y_train.shape[0]+y_val.shape[0] == len(y)
    
    return x_train, y_train, x_val, y_val


x_train, y_train, x_val, y_val = split(x, y, train_split_size)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(609, 21, 151)
(609,)
(177, 21, 151)
(177,)


## Augument Data

In [6]:
# x_aug = copy(x_train)
# y_aug = copy(y_train)

# num_aug=5
# std = 0.02
# for i in range(num_aug):
#     x_perturbed = x_train + np.random.normal(0, std, size=x_train.shape)
#     x_aug = np.concatenate((x_aug, x_perturbed))

#     y_perturbed = y_train + np.random.normal(0, std, size=y_train.shape)
#     y_aug = np.concatenate((y_aug, y_perturbed))
    
# x_aug = x_aug.clip(min=0)
# y_aug = y_aug.clip(min=0)

# x_train = x_aug
# y_train = y_aug


## Model with causal convolutional DNN

In [7]:
K.clear_session()
i = Input(shape=(x.shape[1], x.shape[2]))
print(i.shape)
m = i
m =  Flatten()(m)
m = Dense(500, activation='relu')(m)
m = BatchNormalization()(m)
m = Dense(200, activation='relu')(m)
m = BatchNormalization()(m)
m = Dense(200, activation='relu')(m)
m = Dense(1, activation='relu')(m)

# m = Dense(3000, activation='sigmoid')(m)
# m = Dense(200, activation='relu')(m)
# m = Dense(1, activation='linear')(m)
# m = TimeDistributed(Dense(200, activation='relu'), input_shape=x_train.shape[1:3])(m)

model = Model(inputs=[i], outputs=[m])

# model.summary()
import keras
optimizer=optimizers.Adam(lr=5e-6)
model.compile(optimizer, 'mae')



W0927 21:48:13.584439 140320364234560 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


(?, 21, 151)


Using TensorFlow backend.


In [ ]:
print('Train...')
validation_data = (x_val, y_val)
model.fit(x_train, y_train, epochs=400, verbose=2, validation_data=validation_data, batch_size=50)

Train...
Train on 609 samples, validate on 177 samples
Epoch 1/400
609/609 - 1s - loss: 34.2644 - val_loss: 20.6383
Epoch 2/400
609/609 - 0s - loss: 34.2248 - val_loss: 21.5917
Epoch 3/400
609/609 - 0s - loss: 34.1864 - val_loss: 22.1966
Epoch 4/400
609/609 - 0s - loss: 34.1487 - val_loss: 22.5888
Epoch 5/400
609/609 - 0s - loss: 34.1076 - val_loss: 22.9414
Epoch 6/400
609/609 - 0s - loss: 34.0862 - val_loss: 23.1797
Epoch 7/400
609/609 - 0s - loss: 34.0548 - val_loss: 23.3871
Epoch 8/400
609/609 - 0s - loss: 34.0243 - val_loss: 23.6494
Epoch 9/400
609/609 - 0s - loss: 33.9904 - val_loss: 23.8629
Epoch 10/400
609/609 - 0s - loss: 33.9573 - val_loss: 23.8943
Epoch 11/400
609/609 - 0s - loss: 33.9197 - val_loss: 23.9775
Epoch 12/400
609/609 - 0s - loss: 33.8865 - val_loss: 24.0569
Epoch 13/400
609/609 - 0s - loss: 33.8465 - val_loss: 24.0448
Epoch 14/400
609/609 - 0s - loss: 33.8133 - val_loss: 24.0621
Epoch 15/400
609/609 - 0s - loss: 33.7840 - val_loss: 24.0697
Epoch 16/400
609/609 - 0

Epoch 132/400
609/609 - 0s - loss: 28.2643 - val_loss: 20.5542
Epoch 133/400
609/609 - 0s - loss: 28.2650 - val_loss: 20.5684
Epoch 134/400
609/609 - 0s - loss: 28.1080 - val_loss: 20.6442
Epoch 135/400
609/609 - 0s - loss: 28.0146 - val_loss: 20.8340
Epoch 136/400
609/609 - 0s - loss: 28.1176 - val_loss: 20.6250
Epoch 137/400
609/609 - 0s - loss: 27.9499 - val_loss: 20.6144
Epoch 138/400
609/609 - 0s - loss: 28.0059 - val_loss: 20.5356
Epoch 139/400
609/609 - 0s - loss: 27.7873 - val_loss: 20.5158
Epoch 140/400
609/609 - 0s - loss: 27.8802 - val_loss: 20.2876
Epoch 141/400
609/609 - 0s - loss: 27.7649 - val_loss: 20.3055
Epoch 142/400
609/609 - 0s - loss: 27.8275 - val_loss: 20.4038
Epoch 143/400
609/609 - 0s - loss: 27.7346 - val_loss: 20.4566
Epoch 144/400
609/609 - 0s - loss: 27.6712 - val_loss: 20.3502
Epoch 145/400
609/609 - 0s - loss: 27.5396 - val_loss: 20.3751
Epoch 146/400
609/609 - 0s - loss: 27.6617 - val_loss: 20.3395
Epoch 147/400
609/609 - 0s - loss: 27.5477 - val_loss: 

609/609 - 0s - loss: 22.3756 - val_loss: 17.5698
Epoch 263/400
609/609 - 0s - loss: 21.9896 - val_loss: 17.6269
Epoch 264/400
609/609 - 0s - loss: 22.1960 - val_loss: 17.6681
Epoch 265/400
609/609 - 0s - loss: 22.0107 - val_loss: 17.8943
Epoch 266/400
609/609 - 0s - loss: 21.9011 - val_loss: 17.7614
Epoch 267/400
609/609 - 0s - loss: 22.1529 - val_loss: 17.6425
Epoch 268/400
609/609 - 0s - loss: 21.8241 - val_loss: 17.4884
Epoch 269/400
609/609 - 0s - loss: 21.7060 - val_loss: 17.4482
Epoch 270/400
609/609 - 0s - loss: 22.0824 - val_loss: 17.5117
Epoch 271/400
609/609 - 0s - loss: 22.0721 - val_loss: 17.6729
Epoch 272/400
609/609 - 0s - loss: 22.1642 - val_loss: 17.6086
Epoch 273/400
609/609 - 0s - loss: 21.7501 - val_loss: 17.3082
Epoch 274/400
609/609 - 0s - loss: 21.9110 - val_loss: 17.1847
Epoch 275/400
609/609 - 0s - loss: 22.0453 - val_loss: 17.2703
Epoch 276/400
609/609 - 0s - loss: 21.9758 - val_loss: 17.1112
Epoch 277/400
609/609 - 0s - loss: 21.4833 - val_loss: 17.1255
Epoch 

In [ ]:
p = model.predict(x_train)

plt.plot(p)
plt.plot(y_train)
plt.title('Weekly Dengue Cases')
plt.legend(['predicted', 'actual'])
plt.show()

p = model.predict(x_val)

plt.plot(p)
plt.plot(y_val)
plt.title('Weekly Dengue Cases')
plt.legend(['predicted', 'actual'])
plt.show()

## Forecasting

In [ ]:
def forecast(model, forecast_window, last_train, test_data):
    current = np.expand_dims(last_train, axis=0)
    next_y = model.predict(current)[0][0]
    current[0][0][-1] = next_y
    
    result = [next_y]
    
    
    for xi in test_data:
        xi[0][0:lookback_window] = current[0][0][-lookback_window:]
        current = np.expand_dims(xi, axis=0)
        next_y = model.predict(current)[0][0]
        current[0][0][-1] = next_y
        result.append(next_y)
    
    return result
    
def fake_test_data(x_val):
    x_test = copy(x_val)
    for i in x_test:
        i[0] = 0.0
    return x_test
    
x_test = fake_test_data(x_val)
result = forecast(model, len(x_test), copy(x_train[-200]), x_test)

In [ ]:
print(result)
plt.plot(result)
plt.plot(y_val)
plt.title('Weekly Dengue Cases')
plt.legend(['predicted', 'actual'])
plt.show()
